In [ ]:
import os
os.environ["OPENAI_API_KEY"]="YOUR_API_KEY"

In [2]:
def load_llm():
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) #gpt-4o-mini
        return llm

In [3]:
from langchain.prompts import ChatPromptTemplate

def load_prompt():
        prompt = """ You need to answer the question in the sentence as same as in the  pdf content. . 
        Given below is the context and question of the user.
        context = {context}
        question = {question}
        if the answer is not in the pdf , answer "i donot know what the hell you are asking about"
         """
        prompt = ChatPromptTemplate.from_template(prompt)
        return prompt

In [4]:
#Import Dependencies
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


def load_knowledgeBasee():
        embeddings=OpenAIEmbeddings()
        DB_FAISS_PATH = 'vectorstore/db_faiss'
        db = FAISS.load_local(DB_FAISS_PATH, embeddings,allow_dangerous_deserialization=True)
        return db

In [ ]:
knowledgeBase=load_knowledgeBasee()
llm=load_llm()
prompt=load_prompt()

In [6]:
def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
query = "who is krishna"
if True:
    similar_embeddings=knowledgeBase.similarity_search(query)
    similar_embeddings=FAISS.from_documents(documents=similar_embeddings, embedding=OpenAIEmbeddings())
                
    #creating the chain for integrating llm,prompt,stroutputparser
    retriever = similar_embeddings.as_retriever()
    rag_chain = (
                        {"context": retriever | format_docs, "question": RunnablePassthrough()}
                        | prompt
                        | llm
                        | StrOutputParser()
                    )
                
    response=rag_chain.invoke(query)

In [10]:
response

'Krishna is described as the reservoir of pleasure, the primeval cause of everything, and the cause of all causes in the Brahma-saµhitå. He is also known as Govinda, one who gives pleasure to the senses. Krishna establishes Himself as the source of the material and spiritual worlds, the origin of Brahman and Paramåtmå, and the origin of Nåråya√a in Vaiku√†ha.'